In [45]:
import gensim.models.doc2vec as d2v
# from gensim import models.doc2vec as d2v

In [56]:
# voc_file = open('voc.txt', 'r')

class LabeledLineSentence(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for uid, line in enumerate(open(self.filename)):
            yield d2v.LabeledSentence(words=line.split(), tags=['SENT_%s' % uid])
            
sentences = LabeledLineSentence('voc.txt')

In [61]:
model = d2v.Doc2Vec(sentences, size=100, window=8, min_count=5, workers=4)

In [63]:
model.train(sentences, total_examples=model.corpus_count, epochs=20)

2689757

In [66]:
model.save('test.d2v')

In [67]:
model.infer_vector('Здравствуйте! Мне 21 год, имею условный срок, я хотел бы взять в кредит 150 тыс. дадут ли мне кредит? Скажите пожалуйста')

array([-0.12725984, -0.15446104, -0.36419588,  0.30474874,  0.03359917,
       -0.10460962, -0.10313608,  0.05456538, -0.18159923, -0.16068096,
        0.12578131,  0.03931005,  0.00405481, -0.2957834 , -0.05000006,
       -0.06299557, -0.11241499, -0.02555135,  0.06361082, -0.20107713,
       -0.02974634, -0.09888238, -0.29762021, -0.07791048, -0.0844163 ,
        0.0571858 , -0.00957108, -0.00133738, -0.07083044,  0.33434787,
        0.00791798, -0.158693  ,  0.05915285,  0.1921217 ,  0.11245614,
       -0.23962943,  0.015947  ,  0.26148877, -0.17052972, -0.19314164,
       -0.07718211,  0.26367146, -0.26931611,  0.00859694,  0.00467424,
        0.12746949,  0.19626541, -0.01584804,  0.05746861, -0.2375403 ,
        0.06913897, -0.06733715, -0.23118001,  0.12809303, -0.08924191,
        0.01107481,  0.08951874,  0.05126161,  0.33193687, -0.04705061,
        0.11413155, -0.15052733, -0.05386834, -0.31707594, -0.03734419,
       -0.12740454,  0.14410101, -0.04320796, -0.22567695,  0.11